In [ ]:
# 1. 필수 라이브러리 설치
!pip install fastapi "uvicorn[standard]" pyngrok torch transformers peft pandas accelerate huggingface_hub



In [ ]:
# 2. 파일 업로드
from google.colab import files
import os

print("--- 📂 1/3: serve.py 파일 업로드 ---")
if not os.path.exists('serve.py'):
    files.upload()
else:
    print("serve.py가 이미 존재합니다.")

print("\n--- 📂 2/3: styles.csv 파일 업로드 ---")
if not os.path.exists('styles.csv'):
    files.upload()
else:
    print("styles.csv가 이미 존재합니다.")

print("\n--- 📂 3/3: gemma-fashion-tuner-final.zip 파일 업로드 ---")
if not os.path.exists('gemma-fashion-tuner-final.zip'):
    files.upload()
else:
    print("gemma-fashion-tuner-final.zip이 이미 존재합니다.")


--- 📂 1/3: serve.py 파일 업로드 ---
serve.py가 이미 존재합니다.

--- 📂 2/3: styles.csv 파일 업로드 ---
styles.csv가 이미 존재합니다.

--- 📂 3/3: gemma-fashion-tuner-final.zip 파일 업로드 ---
gemma-fashion-tuner-final.zip이 이미 존재합니다.


In [ ]:
# 3. 모델 폴더 압축 해제
!unzip -q -o gemma-fashion-tuner-final.zip # -o 옵션으로 덮어쓰기 허용


In [ ]:
# 4. 🚨 Hugging Face 로그인 (새로 추가된 가장 중요한 단계)
from huggingface_hub import login

print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
# 이 코드를 실행하면 Hugging Face Access Token을 입력하라는 창이 나타납니다.
login()
print("✅ 로그인 성공!")



--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [ ]:
# 5. 서버를 백그라운드 스레드에서 실행
import threading
import uvicorn
import time
import requests

def run_app():
  # serve.py를 직접 실행하는 대신, 여기서 uvicorn을 실행합니다.
  uvicorn.run("serve:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")



✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...


In [ ]:
# 6. ❗ 서버가 켜질 때까지 똑똑하게 기다리기 (Health Check)
print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다... (최대 10분)")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600: # 최대 10분 대기
    try:
        # Colab 내부에서 서버에 직접 요청을 보내봅니다.
        response = requests.get("http://localhost:8000/", timeout=5)
        # 404 응답이라도 서버가 켜졌다는 신호이므로 성공입니다.
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        # 아직 연결이 거부되면 5초 후 다시 시도합니다.
        time.sleep(5)
        print("...")

if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    # 7. ngrok으로 외부 접속 터널 생성
    from pyngrok import ngrok

    # https://dashboard.ngrok.com/get-started/your-authtoken 에서 토큰을 복사하여 붙여넣으세요.
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)

    ngrok.kill() # 이전에 실행중인 ngrok 터널이 있다면 모두 종료

    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

# Colab 세션이 끝나지 않도록 스레드가 끝날 때까지 대기
thread.join()

⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다... (최대 10분)
...
✅ Colab GPU 환경에서 모델 로딩을 시작합니다...
✅ 사용할 장치: CUDA
...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...
...
...
...
✅ 모델 및 토크나이저 로딩 완료!


INFO:     Started server process [12591]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ styles.csv 로드 완료!
...
INFO:     127.0.0.1:58142 - "GET / HTTP/1.1" 404 Not Found
✅ 서버가 응답하기 시작했습니다!

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://11119ada0da0.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.
INFO:     27.0.238.117:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.186:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.186:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.69:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     183.103.168.167:0 - "OPTIONS /recommend_outfit HTTP/1.1" 405 Method Not Allowed
INFO:     183.103.168.167:0 - "OPTIONS /recommend_outfit HTTP/1.1" 405 Method Not Allowed
INFO:     183.103.168.167:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     211.231.103.92:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.114:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     211.231.103.94:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.69:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.69:0 - "GET 